<a href="https://colab.research.google.com/github/woneuy01/AudioToText/blob/master/Reddit__sentiWordNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import collections
import nltk
import os
nltk.download('wordnet')
import numpy as np
import pandas as pd
#from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn, wordnet as wn
nltk.download('punkt')
nltk.download('sentiwordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [7]:
import glob
import pandas as pd
from pandas import DataFrame
# import multiple cvs file
#path =r'C:\DRO\DCL_rawdata_files' 
#filenames = glob.glob(path + "/*.csv")
filenames = glob.glob("*.csv") # file is in the same directory
print(filenames)
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)
print(big_frame)
#making file as dataframe
df = DataFrame(big_frame,columns=['Unnamed: 0',	'permalink',	'comment',	'subreddit',
                             'subreddit_id',	'total_awards_received',	'created'])
#df.rename(columns={'Unnamed: 0':'index'})
print(df.head())
#stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def tokenize(text):
        tokens = [word for word in nltk.word_tokenize(text)] 
        lemma = [wordnet_lemmatizer.lemmatize(item,pos="v").lower() for item in tokens]
        return lemma


['reddit_comments2020-06-08 00_00_00_2020-06-14 00_00_00.csv', 'reddit_comments2020-06-01 00_00_00_2020-06-07 00_00_00.csv']
      Unnamed: 0  ...              created
0              0  ...  2020-07-03 14:39:56
1              1  ...  2020-07-01 16:03:42
2              2  ...  2020-07-01 14:53:53
3              3  ...  2020-06-19 11:36:07
4              4  ...  2020-06-19 05:30:39
...          ...  ...                  ...
9944        4400  ...       6/4/2020 20:52
9945        1857  ...       6/2/2020 11:50
9946         783  ...       6/6/2020 20:40
9947          16  ...        6/5/2020 2:54
9948        2319  ...       6/6/2020 23:29

[9949 rows x 7 columns]
   Unnamed: 0  ...              created
0           0  ...  2020-07-03 14:39:56
1           1  ...  2020-07-01 16:03:42
2           2  ...  2020-07-01 14:53:53
3           3  ...  2020-06-19 11:36:07
4           4  ...  2020-06-19 05:30:39

[5 rows x 7 columns]


In [4]:
def score_sentiwordnet(doc):   
    synsets = (wn.synsets(w) for w in tokenize(doc) if w is not None) # tokenize and lookup synonyms in WordNet
    words = (s[0].name() for s in synsets if s is not None and len(s) > 0) # use the first (most common) synonym
    words = (swn.senti_synset(s) for s in words) # get sentiment scores which have pos, neg, and neutral parts
    scores = ((1 - w.obj_score()) * (w.pos_score() - w.neg_score()) for w in words) # calculate individual word scores
    return sum(scores)                             #(sum(scores) > 0)

In [15]:
score = [] 
for comment in df['comment']:
  #print(comment)
  pol_score = score_sentiwordnet(comment)
  print(pol_score)
  score.append(pol_score)

df['sentiwordNet']=score

Streaming output truncated to the last 5000 lines.
-0.9375
-1.015625
0.0
0.421875
-0.946625
0.078125
-0.0625
0.359375
-0.78125
0.25
0.765625
0.0
0.28125
0.453125
-0.796875
0.734375
2.640625
-0.265625
-0.328125
0.28125
0.4375
0.0
-0.96875
-0.84375
-0.484375
0.578125
1.265625
0.0
0.765625
0.390625
0.0
0.0625
-0.71875
0.03125
0.3125
-0.6588750000000001
-0.046875
0.0
0.140625
-0.71875
0.125
0.015625
-0.34375
0.046875
-0.90625
0.59375
-0.03125
-1.515625
0.0
0.703125
-0.53125
0.296875
-2.09375
-0.046875
-0.21875
0.375
-0.171875
-0.15625
0.375
-0.53125
-0.296875
-1.046875
-0.734375
-0.53125
0.25
-0.421875
-0.21875
0.46875
0.34375
-0.359375
0.34375
-0.078125
-0.5625
-1.84375
-1.09375
-0.703125
0.390625
-0.515625
-1.171875
-0.171875
0.046875
0.15625
0.0
0.015625
-0.234375
-0.453125
0.140625
0.0
-0.625
-0.75
0.3125
-0.15625
-0.390625
-0.859375
-0.953125
0.78125
-0.09375
0.0625
0.078125
-0.0625
-0.65625
0.125
0.0
-0.265625
0.015625
-0.0625
-0.0625
-0.078125
-0.234375
0.46875
-0.03125
0.765625
0.0

In [17]:
df.head(10)

,Unnamed: 0,permalink,comment,subreddit,subreddit_id,total_awards_received,created,sentiwordNet
0,0,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,"Still waiting on that thousand bucks, buddy.",Coronavirus,t5_2x4yx,0,2020-07-03 14:39:56,-0.234375
1,1,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,[removed],Coronavirus,t5_2x4yx,0,2020-07-01 16:03:42,0.000000
2,2,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,So uhhh.... about that thousand dollars?,Coronavirus,t5_2x4yx,0,2020-07-01 14:53:53,0.000000
3,3,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,They seem ok! \n\nBased on a off hand comment ...,Coronavirus,t5_2x4yx,0,2020-06-19 11:36:07,0.687500
4,4,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,Any news?,Coronavirus,t5_2x4yx,0,2020-06-19 05:30:39,0.000000
5,5,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,"Yeah we all get to choose, that’s totally fine...",Coronavirus,t5_2x4yx,0,2020-06-18 23:12:46,-0.171875
6,6,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,Because I like one more layer in between mysel...,Coronavirus,t5_2x4yx,0,2020-06-18 21:56:23,0.203125
7,7,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,thats some bull,Coronavirus,t5_2x4yx,0,2020-06-18 16:19:50,0.000000
8,8,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,maybe thats why they have no problem giving ba...,Coronavirus,t5_2x4yx,0,2020-06-18 16:19:27,-1.203125
9,9,/r/Coronavirus/comments/h0p2bs/its_serious_sou...,Either that or american education,Coronavirus,t5_2x4yx,0,2020-06-18 03:38:53,0.062500


In [18]:
df.to_csv("reddit_comments"+"*"+".csv",encoding="utf-8-sig")